# (Optional) Prompty

Microsoft Prompty is a tool designed to help developers create, manage, and evaluate prompts for LLMs more efficiently. It works within the VSCode environment and is especially useful for refining AI interactions in GenAI pplications. <br>
Prompty provides a standardized format (using markdown) for defining prompts, making it easier to understand, share, and debug. It allows developers to quickly prototype

-   Reference: https://microsoft.github.io/promptflow/tutorials/prompty-quickstart.html


### Start Trace


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
module_path = "../../0_lab_preparation"
sys.path.append(os.path.abspath(module_path))

from common import check_kernel
check_kernel()

In [ ]:
from promptflow.tracing import start_trace
from dotenv import load_dotenv, find_dotenv
load_dotenv()

# start a trace session, and print a url for user to check trace
start_trace()

### Connection override


In [ ]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration

# override configuration with AzureOpenAIModelConfiguration
configuration = AzureOpenAIModelConfiguration(
    azure_endpoint="${env:AZURE_OPENAI_ENDPOINT}",  # Use ${env:<ENV_NAME>} to surround the environment variable name.
    api_key="${env:AZURE_OPENAI_API_KEY}",
    api_version="${env:AZURE_OPENAI_API_VERSION}",
    azure_deployment="gpt-4o-mini",
)

override_model = {"configuration": configuration, "parameters": {"max_tokens": 512}}

### Basic prompt


In [ ]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load(source="./prompty/basic.prompty", model=override_model)

# execute the flow as function
result = f(question="What is the capital of Japan?")
result

In [ ]:
f = Prompty.load(source="./prompty/japanese.prompty", model=override_model)

context = """
アルパインエクスプローラーテントには、プライバシーを確保するために取り外し可能な仕切りがあります。
多数のメッシュ窓と換気用の調節可能な通気口、および
防水設計が特徴です。アウトドアの必需品を収納するためのギアロフトも組み込まれています
機器ロフトが組み込まれています。要するに、プライバシー、快適さ、
便利さの組み合わせは、自然の中でそれを第二の家にします!
"""
result = f(firstName="Hyo", context=context, question="テントについて何を知りたいですか?")
result

### Evaluation


In [ ]:
flow = Prompty.load(source="./prompty/basic.prompty", model=override_model)
eval_flow = Prompty.load("./prompty/eval.prompty", model=override_model)

question = "What is the capital of Japan?"
ground_truth = "Tokyo"

result = flow(question=question)
eval_result = eval_flow(question=question, ground_truth=ground_truth, answer=result)

print(f"result: {result}")
print(f"eval_result: {eval_result}")

### Chat


In [ ]:
LANGUAGE = "Japanese"
f = Prompty.load("./prompty/chat.prompty", model=override_model)

chat_history = [
    {"role": "system", "content": "You are a factual chatbot that is also sarcastic."}, 
    {"role": "user", "content": "How far is the Moon from Earth?"}, 
    {"role": "assistant", "content": "384,400 kilometers"},
    {"role": "user", "content": "Can you be more sarcastic?"}, 
    {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}
]
question = f"Can you speak more scientifically in {LANGUAGE}?"
result = f(chat_history=chat_history, question=question)
print(result)